## Building Pipeline

In [3]:
import tensorflow as tf
import os

import cv2
import imghdr

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
batch_size = 32

In [5]:
image_generator = ImageDataGenerator(rescale = 1./255, shear_range = 0.1, zoom_range = 0.1)

test_generator = ImageDataGenerator(rescale = 1./255)

In [6]:
train_data = image_generator.flow_from_directory('chest_xray/train', 
                                                 class_mode='binary', 
                                                 batch_size = batch_size, 
                                                 classes = ['NORMAL', 'PNEUMONIA'], 
                                                 color_mode = 'grayscale', 
                                                 shuffle=False, 
                                                 target_size=(256, 256))

val_data = test_generator.flow_from_directory('chest_xray/val', 
                                              class_mode='binary', 
                                              batch_size = batch_size, 
                                              classes = ['NORMAL', 'PNEUMONIA'], 
                                              color_mode = 'grayscale', 
                                              shuffle=False, 
                                              target_size=(256, 256))

test_data = test_generator.flow_from_directory('chest_xray/test', 
                                               class_mode='binary', 
                                               batch_size = batch_size, 
                                               classes = ['NORMAL', 'PNEUMONIA'], 
                                               color_mode = 'grayscale', 
                                               shuffle=False, 
                                               target_size=(256, 256))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
tf.random.shuffle(train_data.classes)

<tf.Tensor: shape=(5216,), dtype=int32, numpy=array([0, 1, 1, ..., 0, 1, 1])>

In [8]:
class_names = train_data.classes
class_names

array([0, 0, 0, ..., 1, 1, 1])

## Label Shuffle

In [7]:
import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Rescaling

In [13]:
train_data.next()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [8]:
data_list = []
batch_index = 0

while batch_index <= train_data.batch_index:
    data = train_data.next()
    data_list.append(data[0])
    print(batch_index)
    batch_index = batch_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162


In [32]:
train_data_new = data_list[0]

In [36]:
print(train_data_new[0])

[[[0.0627451 ]
  [0.12787947]
  [0.19824085]
  ...
  [0.37490547]
  [0.36672753]
  [0.35928303]]

 [[0.0627451 ]
  [0.12789483]
  [0.19825402]
  ...
  [0.374904  ]
  [0.36672536]
  [0.35928667]]

 [[0.06317858]
  [0.12087499]
  [0.19224645]
  ...
  [0.3764266 ]
  [0.369469  ]
  [0.36231443]]

 ...

 [[0.16571997]
  [0.16729784]
  [0.16777998]
  ...
  [0.40001008]
  [0.40782538]
  [0.40332448]]

 [[0.18382895]
  [0.18102622]
  [0.18353043]
  ...
  [0.4352464 ]
  [0.44460127]
  [0.43529415]]

 [[0.1838297 ]
  [0.18102548]
  [0.18353117]
  ...
  [0.43524933]
  [0.44459903]
  [0.43529415]]]


## Fir model

In [43]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

In [37]:
model = Sequential()

In [38]:
model.add(Conv2D(8, (3,3), 1, activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

In [39]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy', 'AUC'])

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 8)       80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 32)       0

In [44]:
unique_classes = np.unique(train_data.classes)
classes = train_data.classes

weights = compute_class_weight(class_weight = 'balanced', classes= unique_classes, y = classes)

class_weights = dict(zip( np.unique(train_data.classes), weights))

In [46]:
hist = model.fit(train_data_new, epochs=20, validation_data = val_data, class_weight = class_weights)

AttributeError: 'NoneType' object has no attribute 'shape'